In [ ]:
using Pkg
pkg"activate ."

In [ ]:
# Run this cell in order to download all the package dependencies with the exact versions used in the book
# This is necessary if (some of) the packages have been updated and have introduced breaking changes
pkg"instantiate"

In [ ]:
using CSV

In [ ]:
users = CSV.read("data/BX-Users.csv", header = 1, delim = ';', missingstring = "NULL", escapechar = '\\')

In [ ]:
books = CSV.read("data/BX-Books.csv", header = 1, delim = ';', missingstring = "NULL", escapechar = '\\')

In [ ]:
books_ratings = CSV.read("data/BX-Book-Ratings.csv", header = 1, delim = ';', missingstring = "NULL", escapechar = '\\')

In [ ]:
using DataFrames

In [ ]:
describe(users, stats = [:min, :max, :nmissing, :nunique, :eltype])

In [ ]:
using Gadfly

In [ ]:
plot(users, x = :Age, Geom.histogram(bincount = 15))

In [ ]:
users = users[users[:Age] .< 100, :]

In [ ]:
using Statistics

In [ ]:
mean(skipmissing(users[:Age]))

In [ ]:
users[:Age] = coalesce.(users[:Age], mean(skipmissing(users[:Age])))

In [ ]:
users = users[users[:Age] .< 100, :]

In [ ]:
head(users)

In [ ]:
describe(books, stats = [:nmissing, :nunique, :eltype])

In [ ]:
maximum(skipmissing(books[Symbol("Year-Of-Publication")]))

In [ ]:
minimum(skipmissing(books[Symbol("Year-Of-Publication")]))

In [ ]:
plot(books, x = Symbol("Year-Of-Publication"), Geom.histogram)

In [ ]:
unique(books[Symbol("Year-Of-Publication")]) |> sort

In [ ]:
books = books[books[Symbol("Year-Of-Publication")] .>= 1970, :]

In [ ]:
books = books[books[Symbol("Year-Of-Publication")] .<= 2004, :]

In [ ]:
plot(books, x = Symbol("Year-Of-Publication"), Geom.histogram)

In [ ]:
describe(books_ratings)

In [ ]:
plot(books_ratings, x = Symbol("Book-Rating"), Geom.histogram)

In [ ]:
books_ratings = books_ratings[books_ratings[Symbol("Book-Rating")] .> 0, :]

In [ ]:
plot(books_ratings, x = Symbol("Book-Rating"), Geom.histogram)

In [ ]:
books_ratings_books = join(books_ratings, books, on = :ISBN, kind = :inner)

In [ ]:
books_ratings_books_users = join(books_ratings_books, users, on = Symbol("User-ID"), kind = :inner)

In [ ]:
top_ratings = books_ratings_books_users[books_ratings_books_users[Symbol("Book-Rating")] .>= 8, :]

In [ ]:
for n in names(top_ratings)
    rename!(top_ratings, n => Symbol(replace(string(n), "-"=>"")))
end

In [ ]:
names(top_ratings)

In [ ]:
ratings_count = by(top_ratings, :UserID, df -> size(df[:UserID])[1])

In [ ]:
describe(ratings_count)

In [ ]:
ratings_count = ratings_count[ratings_count[:x1] .>= 5, :]

In [ ]:
plot(ratings_count, x = :x1, Geom.histogram(maxbincount = 6))

In [ ]:
ratings_count[ratings_count[:x1] .> 1000, :]

In [ ]:
ratings_count = ratings_count[ratings_count[:x1] .<= 1000, :]

In [ ]:
top_ratings = join(top_ratings, ratings_count, on = :UserID, kind = :inner)

In [ ]:
CSV.write("top_ratings.csv", top_ratings)